In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import sys
sys.path.append('../')
from tqdm import trange
from fact_check.constants import DATA_DIR, cache_dir

In [2]:
df = pd.read_csv(DATA_DIR/'semmeddb/semmeddb_processed_10.csv')

/tmp/ipykernel_2700472/1302764665.py:1: DtypeWarning: Columns (4,8,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_DIR/'semmeddb/semmeddb_processed_10.csv')


In [3]:
def clean_predicate(st):
    st = st.replace('_', ' ')
    if st == "ISA":
        return 'IS A'
    return st

df['predicate_clean'] = df['PREDICATE'].apply(clean_predicate)

In [4]:
df['OBJECT_NAME'] = df['OBJECT_NAME'].fillna(0.)
df['OBJECT_NAME'] = df['OBJECT_NAME'].astype(str)
df['SUBJECT_NAME'] = df['SUBJECT_NAME'].astype(str)

In [5]:
df['sent_str'] = df.apply(lambda x: x['SUBJECT_NAME'] + ' ' + x['predicate_clean'] + ' ' + x['OBJECT_NAME'], axis = 1)

In [12]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def process_batch(sentences_batch):
    encoded_input = tokenizer(sentences_batch, padding=True, truncation=True, return_tensors='pt')
    for i in encoded_input:
        encoded_input[i] = encoded_input[i].to('cuda')    
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)    
    return sentence_embeddings

# Sentences we want sentence embeddings for
sentences = list(df['sent_str'].values)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to('cuda')

In [12]:
batch_size = 1024
embs = []

for i in trange(0, len(sentences), batch_size):
    batch = sentences[i:i+batch_size]
    embs.append(process_batch(batch).detach().cpu().numpy())                          

100%|██████████████████| 7060/7060 [14:17<00:00,  8.24it/s]


In [13]:
embs = np.concatenate(embs, axis=0)

In [14]:
embs.shape

(7229041, 384)

In [15]:
np.save(cache_dir/'kg_embeddings.npy', embs)